# 01_dataset_overview.ipynb

Файл делает обзор датасета CodeSearchNet для Python и подготавливает данные для следующих этапов. Загружается и анализируется структура датасета, настраивается tree-sitter для синтаксического разбора кода, извлекаются имена функций и их тела. Формируются два варианта представления кода: с документацией + комментариями и без них


### Инициализация окружения и проверка датасета CodeSearchNet

Проверить корректность окружения, доступность датасета CodeSearchNet для языка Python и изучить его структуру перед дальнейшей обработкой

In [68]:
import sys
import platform

print("Python version:", sys.version)
print("Platform:", platform.platform())

Python version: 3.10.0 (tags/v3.10.0:b494f59, Oct  4 2021, 19:00:18) [MSC v.1929 64 bit (AMD64)]
Platform: Windows-10-10.0.19045-SP0


In [69]:
from datasets import load_dataset

In [70]:
# Загрузка test-сплита CodeSearchNet для языка Python
# Важно: trust_remote_code=True
ds = load_dataset(
    "code_search_net",
    "python",
    split="test",
    trust_remote_code=True,
)

print("Полный размер test-сплита:", len(ds))

Полный размер test-сплита: 22176


In [71]:
# Ограничиваемся первыми 1000 примерами, как указано в задании
ds_small = ds.select(range(1000))

print("Размер ds_small:", len(ds_small))


Размер ds_small: 1000


In [72]:
# Названия полей датасета
ds_small.column_names


['repository_name',
 'func_path_in_repository',
 'func_name',
 'whole_func_string',
 'language',
 'func_code_string',
 'func_code_tokens',
 'func_documentation_string',
 'func_documentation_tokens',
 'split_name',
 'func_code_url']

In [73]:
example = ds_small[0]

print("func_name:")
print(example["func_name"])
print("\nfunc_documentation_string:")
print(example["func_documentation_string"])
print("\nwhole_func_string (обрезано):")
print(example["whole_func_string"][:500])


func_name:
YouTube.get_vid_from_url

func_documentation_string:
Extracts video ID from URL.

whole_func_string (обрезано):
def get_vid_from_url(url):
        """Extracts video ID from URL.
        """
        return match1(url, r'youtu\.be/([^?/]+)') or \
          match1(url, r'youtube\.com/embed/([^/?]+)') or \
          match1(url, r'youtube\.com/v/([^/?]+)') or \
          match1(url, r'youtube\.com/watch/([^/?]+)') or \
          parse_query_param(url, 'v') or \
          parse_query_param(parse_query_param(url, 'u'), 'v')


In [74]:
# Проверим, что функция реально содержится в whole_func_string
print("Имя функции встречается в коде:",
      example["func_name"].split(".")[-1] in example["whole_func_string"])


Имя функции встречается в коде: True


### Выводы

Датасет CodeSearchNet для Python загружается без проблем и содержит код функций, их имена и документацию. Поле `func_name` не всегда подходит для прямого использования, поэтому имя функции требуется извлекать из AST


## Анализ структуры Python-кода в CodeSearchNet

Посмотреть, как реально выглядят функции в датасете CodeSearchNet для Python. Понять, с какими типами функций приходится работать и почему для разбора нельзя использовать простые строковые методы

In [75]:
import random

def print_example(idx):
    ex = ds_small[idx]
    print("=" * 80)
    print(f"Index: {idx}")
    print("func_name:", ex["func_name"])
    print("Documentation:", ex["func_documentation_string"])
    print("Code (обрезано):")
    print(ex["whole_func_string"][:600])

# Посмотрим несколько случайных примеров
for idx in random.sample(range(len(ds_small)), 3):
    print_example(idx)


Index: 239
func_name: CloudNaturalLanguageHook.get_conn
Documentation: Retrieves connection to Cloud Natural Language service.

        :return: Cloud Natural Language service object
        :rtype: google.cloud.language_v1.LanguageServiceClient
Code (обрезано):
def get_conn(self):
        """
        Retrieves connection to Cloud Natural Language service.

        :return: Cloud Natural Language service object
        :rtype: google.cloud.language_v1.LanguageServiceClient
        """
        if not self._conn:
            self._conn = LanguageServiceClient(credentials=self._get_credentials())
        return self._conn
Index: 355
func_name: TaskInstance.ready_for_retry
Documentation: Checks on whether the task instance is in the right state and timeframe
        to be retried.
Code (обрезано):
def ready_for_retry(self):
        """
        Checks on whether the task instance is in the right state and timeframe
        to be retried.
        """
        return (self.state == State.UP_FO

In [76]:
def find_class_methods(dataset, limit=5):
    results = []
    for ex in dataset:
        if "." in ex["func_name"]:
            results.append(ex)
        if len(results) >= limit:
            break
    return results

class_methods = find_class_methods(ds_small)

for ex in class_methods:
    print("=" * 80)
    print("Qualified func_name:", ex["func_name"])
    print(ex["whole_func_string"][:400])


Qualified func_name: YouTube.get_vid_from_url
def get_vid_from_url(url):
        """Extracts video ID from URL.
        """
        return match1(url, r'youtu\.be/([^?/]+)') or \
          match1(url, r'youtube\.com/embed/([^/?]+)') or \
          match1(url, r'youtube\.com/v/([^/?]+)') or \
          match1(url, r'youtube\.com/watch/([^/?]+)') or \
          parse_query_param(url, 'v') or \
          parse_query_param(parse_query_param(url, 
Qualified func_name: BokeCC.download_by_id
def download_by_id(self, vid = '', title = None, output_dir='.', merge=True, info_only=False,**kwargs):
        """self, str->None
        
        Keyword arguments:
        self: self
        vid: The video ID for BokeCC cloud, something like
        FE3BB999594978049C33DC5901307461
        
        Calls the prepare() to download the video.
        
        If no title is provided, this method 
Qualified func_name: QiE.get_vid_from_url
def get_vid_from_url(self, url):
        """Extracts video ID from 

In [77]:
def find_async_functions(dataset, limit=3):
    results = []
    for ex in dataset:
        if ex["whole_func_string"].lstrip().startswith("async def"):
            results.append(ex)
        if len(results) >= limit:
            break
    return results

async_funcs = find_async_functions(ds_small)

for ex in async_funcs:
    print("=" * 80)
    print("func_name:", ex["func_name"])
    print(ex["whole_func_string"][:400])


In [78]:
def find_decorated_functions(dataset, limit=3):
    results = []
    for ex in dataset:
        if ex["whole_func_string"].lstrip().startswith("@"):
            results.append(ex)
        if len(results) >= limit:
            break
    return results

decorated_funcs = find_decorated_functions(ds_small)

for ex in decorated_funcs:
    print("=" * 80)
    print("func_name:", ex["func_name"])
    print(ex["whole_func_string"][:400])


In [79]:
def find_nested_functions(dataset, limit=3):
    results = []
    for ex in dataset:
        code = ex["whole_func_string"]
        if code.count("def ") > 1:
            results.append(ex)
        if len(results) >= limit:
            break
    return results

nested_funcs = find_nested_functions(ds_small)

for ex in nested_funcs:
    print("=" * 80)
    print("func_name:", ex["func_name"])
    print(ex["whole_func_string"][:500])


func_name: get_video_url_from_video_id
def get_video_url_from_video_id(video_id):
    """Splicing URLs according to video ID to get video details"""
    # from js
    data = [""] * 256
    for index, _ in enumerate(data):
        t = index
        for i in range(8):
            t = -306674912 ^ unsigned_right_shitf(t, 1) if 1 & t else unsigned_right_shitf(t, 1)
        data[index] = t

    def tmp():
        rand_num = random.random()
        path = "/video/urls/v/1/toutiao/mp4/{video_id}?r={random_num}".format(video_id=video_id,
   
func_name: restart_workers
def restart_workers(gunicorn_master_proc, num_workers_expected, master_timeout):
    """
    Runs forever, monitoring the child processes of @gunicorn_master_proc and
    restarting workers occasionally.
    Each iteration of the loop traverses one edge of this state transition
    diagram, where each state (node) represents
    [ num_ready_workers_running / num_workers_running ]. We expect most time to
    be spent in [n / n]. `

In [80]:
def analyze_docs_and_comments(dataset):
    with_doc = 0
    with_comments = 0

    for ex in dataset:
        code = ex["whole_func_string"]
        if '"""' in code or "'''" in code:
            with_doc += 1
        if "#" in code:
            with_comments += 1

    return with_doc, with_comments

doc_count, comment_count = analyze_docs_and_comments(ds_small)

print(f"Функций с docstring: {doc_count} / {len(ds_small)}")
print(f"Функций с комментариями: {comment_count} / {len(ds_small)}")


Функций с docstring: 1000 / 1000
Функций с комментариями: 368 / 1000


### Выводы по анализу структуры кода

В датасете встречаются методы классов, асинхронные функции, декораторы и вложенные функции. Поле `func_name` часто содержит квалифицированное имя, тогда как в коде объявлено только имя функции. Для разбора такой структуры нужно сделать синтаксический анализ, а не строковые методы

## Установка и запуск tree-sitter для Python

Настроить tree-sitter для разбора Python-кода. Проверить, что парсер корректно работает на данных из CodeSearchNet.
Подготовить базовый код, который будет использоваться дальше при извлечении имени функции и её тела

In [81]:
# !pip install tree-sitter tree-sitter-python

In [82]:
from tree_sitter import Language, Parser
import tree_sitter_python

print("tree-sitter-python module loaded:", tree_sitter_python)


tree-sitter-python module loaded: <module 'tree_sitter_python' from 'D:\\Ai_Lab_2\\02-func-name-suggestion\\.venv\\lib\\site-packages\\tree_sitter_python\\__init__.py'>


In [83]:
# Получаем язык Python
PY_LANGUAGE = Language(tree_sitter_python.language())

# Создаём парсер
parser = Parser()
parser.language = PY_LANGUAGE

print("Tree-sitter parser for Python initialized")


Tree-sitter parser for Python initialized


In [84]:
# Берём пример функции из датасета
test_code = ds_small[0]["whole_func_string"]

# Парсим исходный код
tree = parser.parse(bytes(test_code, "utf8"))

# Корень синтаксического дерева
root_node = tree.root_node

print("Root node type:", root_node.type)

Root node type: module


In [85]:
for child in root_node.children:
    print(child.type)


function_definition


In [86]:
def find_function_definitions(node):
    results = []
    if node.type in ("function_definition", "async_function_definition"):
        results.append(node)
    for child in node.children:
        results.extend(find_function_definitions(child))
    return results

func_defs = find_function_definitions(root_node)

print("Found function definitions:", len(func_defs))
for fn in func_defs:
    print("Function node type:", fn.type)


Found function definitions: 1
Function node type: function_definition


In [87]:
def extract_function_name(func_node, source_code):
    for child in func_node.children:
        if child.type == "identifier":
            return source_code[child.start_byte:child.end_byte]
    return None

source_bytes = bytes(test_code, "utf8")

for fn in func_defs:
    name = extract_function_name(fn, source_bytes)
    print("Extracted name:", name)


Extracted name: b'get_vid_from_url'


### Выводы по настройке tree-sitter

Tree-sitter корректно разбирает Python-код из CodeSearchNet. С AST можно находить определения функций и извлекать их имена напрямую, без анализа строк

## Извлечение тела функции из AST

Определить, где именно в AST находится тело функции, и научиться извлекать его целиком, без поиска по строкам и регулярным выражениям

In [88]:
def get_function_body_node(func_node):
    """
    Возвращает узел тела функции (block) для function_definition или
    async_function_definition.
    """
    for child in func_node.children:
        if child.type == "block":
            return child
    return None

In [89]:
def extract_function_body(func_node, source_bytes):
    """
    Извлекает текст тела функции по байтовым границам узла block.
    """
    body_node = get_function_body_node(func_node)
    if body_node is None:
        return None

    return source_bytes[body_node.start_byte:body_node.end_byte]


In [90]:
source_bytes = bytes(test_code, "utf8")

for fn in func_defs:
    body = extract_function_body(fn, source_bytes)
    print("Extracted function body:")
    print(body.decode("utf8"))


Extracted function body:
"""Extracts video ID from URL.
        """
        return match1(url, r'youtu\.be/([^?/]+)') or \
          match1(url, r'youtube\.com/embed/([^/?]+)') or \
          match1(url, r'youtube\.com/v/([^/?]+)') or \
          match1(url, r'youtube\.com/watch/([^/?]+)') or \
          parse_query_param(url, 'v') or \
          parse_query_param(parse_query_param(url, 'u'), 'v')


In [91]:
body_text = body.decode("utf8")

print("Starts with indentation:", body_text.lstrip() != body_text)
print("Contains return:", "return" in body_text)


Starts with indentation: False
Contains return: True


### Выводы по извлечению тела функции

Тело функции извлекается напрямую из AST через узел `block`. В результате получается исходный код функции с сохранённой структурой и форматированием

## Извлечение тела функции без комментариев и документации

Получить тело функции, в котором удалены документация и комментарии, используя структуру AST, а не строковый разбор

In [92]:
def is_docstring_node(node):
    """
    Проверяет, является ли узел docstring'ом:
    expression_statement со строковым литералом.
    """
    if node.type != "expression_statement":
        return False

    if len(node.children) == 0:
        return False

    return node.children[0].type == "string"


In [93]:
def extract_function_body_no_comments(func_node, source_bytes):
    """
    Извлекает тело функции без docstring и комментариев.
    """
    body_node = get_function_body_node(func_node)
    if body_node is None:
        return None

    cleaned_chunks = []
    docstring_skipped = False

    for child in body_node.children:
        # Пропускаем комментарии
        if child.type == "comment":
            continue

        # Пропускаем docstring (только первый)
        if not docstring_skipped and is_docstring_node(child):
            docstring_skipped = True
            continue

        cleaned_chunks.append(
            source_bytes[child.start_byte:child.end_byte]
        )

    return b"\n".join(cleaned_chunks)


In [94]:
for fn in func_defs:
    body_no_comments = extract_function_body_no_comments(fn, source_bytes)
    print("Function body without comments and docstring:")
    print(body_no_comments.decode("utf8"))


Function body without comments and docstring:
return match1(url, r'youtu\.be/([^?/]+)') or \
          match1(url, r'youtube\.com/embed/([^/?]+)') or \
          match1(url, r'youtube\.com/v/([^/?]+)') or \
          match1(url, r'youtube\.com/watch/([^/?]+)') or \
          parse_query_param(url, 'v') or \
          parse_query_param(parse_query_param(url, 'u'), 'v')


In [95]:
body_text = body_no_comments.decode("utf8")

print("Contains docstring:", '"""' in body_text or "'''" in body_text)
print("Contains comments:", "#" in body_text)
print("Contains return:", "return" in body_text)


Contains docstring: False
Contains comments: False
Contains return: True


### Выводы по удалению комментариев и docstring

Docstring и комментарии корректно удаляются на уровне AST. Полученное тело функции содержит только исполняемый код и подходит для использования без дополнительной семантической информации

## Формирование итогового датасета

Собрать итоговый датасет на базе CodeSearchNet для Python, в котором для каждой функции сохранены имя, тело без комментариев и docstring, а также тело с документацией и комментариями


In [96]:
from datasets import load_dataset

ds = load_dataset(
    "code_search_net",
    "python",
    split="test",
    trust_remote_code=True,
)

ds = ds.select(range(1000))
print("Dataset size:", len(ds))


Dataset size: 1000


In [97]:
from tree_sitter import Language, Parser
import tree_sitter_python

PY_LANGUAGE = Language(tree_sitter_python.language())

parser = Parser()
parser.language = PY_LANGUAGE


In [98]:
def process_example(example):
    code = example["whole_func_string"]
    source_bytes = bytes(code, "utf8")

    tree = parser.parse(source_bytes)
    root = tree.root_node

    func_defs = find_function_definitions(root)
    if len(func_defs) == 0:
        return {
            "parsed_func_name": None,
            "body_no_comments": None,
            "body_with_comments": None,
        }

    func_node = func_defs[0]

    func_name = extract_function_name(func_node, source_bytes)
    body_with_comments = extract_function_body(func_node, source_bytes)
    body_no_comments = extract_function_body_no_comments(func_node, source_bytes)

    return {
        "parsed_func_name": func_name.decode("utf8") if func_name else None,
        "body_no_comments": body_no_comments.decode("utf8") if body_no_comments else None,
        "body_with_comments": body_with_comments.decode("utf8") if body_with_comments else None,
    }


In [99]:
ds_processed = ds.map(process_example)


Map: 100%|██████████| 1000/1000 [00:00<00:00, 4864.99 examples/s]


In [100]:
ds_processed.column_names


['repository_name',
 'func_path_in_repository',
 'func_name',
 'whole_func_string',
 'language',
 'func_code_string',
 'func_code_tokens',
 'func_documentation_string',
 'func_documentation_tokens',
 'split_name',
 'func_code_url',
 'parsed_func_name',
 'body_no_comments',
 'body_with_comments']

In [101]:
sample = ds_processed[0]

print("Original func_name:", sample["func_name"])
print("Parsed func_name:", sample["parsed_func_name"])

print("\nBody without comments:")
print(sample["body_no_comments"][:300])

print("\nBody with comments:")
print(sample["body_with_comments"][:300])


Original func_name: YouTube.get_vid_from_url
Parsed func_name: get_vid_from_url

Body without comments:
return match1(url, r'youtu\.be/([^?/]+)') or \
          match1(url, r'youtube\.com/embed/([^/?]+)') or \
          match1(url, r'youtube\.com/v/([^/?]+)') or \
          match1(url, r'youtube\.com/watch/([^/?]+)') or \
          parse_query_param(url, 'v') or \
          parse_query_param(parse_que

Body with comments:
"""Extracts video ID from URL.
        """
        return match1(url, r'youtu\.be/([^?/]+)') or \
          match1(url, r'youtube\.com/embed/([^/?]+)') or \
          match1(url, r'youtube\.com/v/([^/?]+)') or \
          match1(url, r'youtube\.com/watch/([^/?]+)') or \
          parse_query_param(u


In [102]:
ds_final = ds_processed.filter(
    lambda x: x["parsed_func_name"] is not None
              and x["body_no_comments"] is not None
              and x["body_with_comments"] is not None
)

print("Final dataset size:", len(ds_final))


Filter: 100%|██████████| 1000/1000 [00:00<00:00, 12606.75 examples/s]

Final dataset size: 1000


In [103]:
import os

os.makedirs("artifacts", exist_ok=True)
ds_final.save_to_disk("artifacts/ds_python_1000_prepared")

print("Saved to artifacts/ds_python_1000_prepared")


Saving the dataset (1/1 shards): 100%|██████████| 1000/1000 [00:00<00:00, 68099.30 examples/s]

Saved to artifacts/ds_python_1000_prepared


## Файл готов и можно начинать работать